# CS 145 COVID Prediction

## Loading Data

First we need to load the data, which is stored in CSV files separated by date

In [45]:
import numpy as np
import pandas as pd
import glob
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

In [46]:
# grab filepaths for every csv file
datapath = 'data/csse_covid_19_daily_reports_us/*.csv'
datafiles = np.array(glob.glob(datapath))
print(datafiles[138:142])

['data/csse_covid_19_daily_reports_us\\08-28-2020.csv'
 'data/csse_covid_19_daily_reports_us\\08-29-2020.csv'
 'data/csse_covid_19_daily_reports_us\\08-30-2020.csv'
 'data/csse_covid_19_daily_reports_us\\08-31-2020.csv']


In [47]:
# init giant df
data = pd.read_csv(datafiles[0])
# read and append each csv to the master df
for i in range(1, len(datafiles)):
    df = pd.read_csv(datafiles[i])
    data = data.append(df, ignore_index = True)
# only keep 50 states
data.drop(data[data.Province_State.isin(['Diamond Princess', 'District of Columbia', 'Grand Princess', 'Guam', 'Puerto Rico', 'American Samoa', 'Northern Mariana Islands', 'Recovered', 'Virgin Islands'])].index, inplace = True)
data.drop(columns = ['Country_Region', 'FIPS', 'UID', 'ISO3'], inplace = True)
data['Last_Update'] = data['Last_Update'].astype('datetime64[ns]')
data = data.reset_index(drop=True)
data.loc[48:52, ]

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
48,Wisconsin,2020-04-12 23:18:15,44.2685,-89.6165,3341,144,NaN,3197.0,64.565739,39257.0,974.0,4.310087,758.652263,29.152948
49,Wyoming,2020-04-12 23:18:15,42.7560,-107.3025,270,0,137.0,270.0,54.299735,5459.0,40.0,0.000000,1097.860188,14.814815
50,Alabama,2020-04-13 23:07:54,32.3182,-86.9023,3734,99,NaN,3635.0,79.634933,29182.0,457.0,2.651312,622.363852,12.238886
51,Alaska,2020-04-13 23:07:54,61.3707,-152.4044,277,8,85.0,269.0,46.340521,7830.0,32.0,2.888087,1309.914362,11.552347
52,Arizona,2020-04-13 23:07:54,33.7298,-111.4312,3705,122,NaN,3583.0,50.901828,43347.0,525.0,3.292848,595.530778,14.170040


## Data Cleaning

In [48]:
data = pd.concat([data, pd.get_dummies(data['Province_State'])], axis = 1)
statenames = np.unique(data['Province_State'])
#data.drop(columns = ['Province_State'], inplace = True)
names = list(data.columns.values)
timestamps = data['Last_Update']
states = data['Province_State']
data.head()

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alabama,2020-04-12 23:18:15,32.3182,-86.9023,3563,93,NaN,3470.0,75.988020,21583.0,...,0,0,0,0,0,0,0,0,0,0
1,Alaska,2020-04-12 23:18:15,61.3707,-152.4044,272,8,66.0,264.0,45.504049,8038.0,...,0,0,0,0,0,0,0,0,0,0
2,Arizona,2020-04-12 23:18:15,33.7298,-111.4312,3542,115,NaN,3427.0,48.662422,42109.0,...,0,0,0,0,0,0,0,0,0,0
3,Arkansas,2020-04-12 23:18:15,34.9697,-92.3731,1280,27,367.0,1253.0,49.439423,19722.0,...,0,0,0,0,0,0,0,0,0,0
4,California,2020-04-12 23:18:15,36.1162,-119.6816,22795,640,NaN,22155.0,58.137726,190328.0,...,0,0,0,0,0,0,0,0,0,0


We impute NA values using KNN with k = 10.

In [49]:
imputer = KNNImputer(n_neighbors=10, weights="uniform")
data = pd.DataFrame(data = imputer.fit_transform(data.drop(columns = ['Province_State', 'Last_Update'])), columns = names[2:])
data = pd.concat([states, timestamps, data], axis = 1)
data.head()

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alabama,2020-04-12 23:18:15,32.3182,-86.9023,3563.0,93.0,780.3,3470.0,75.988020,21583.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaska,2020-04-12 23:18:15,61.3707,-152.4044,272.0,8.0,66.0,264.0,45.504049,8038.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Arizona,2020-04-12 23:18:15,33.7298,-111.4312,3542.0,115.0,1378.2,3427.0,48.662422,42109.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arkansas,2020-04-12 23:18:15,34.9697,-92.3731,1280.0,27.0,367.0,1253.0,49.439423,19722.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,California,2020-04-12 23:18:15,36.1162,-119.6816,22795.0,640.0,4283.8,22155.0,58.137726,190328.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Additional Features

We will hard code each state's overall political stance in 2020, with 3 indicating safe, 2 indicating likely, 1 indicating leaning, and 0 indicating none. Thus the parameters "dem" and "rep" are ordinal while "swing" is a dummy variable. Because Maine and Nebraska allow split voting, we will consider Maine likely democratic and Nebraska likely republican. The source of the data is from https://www.270towin.com/

We will also include whether a state has enacted a mandatory mask mandate, the data for which is from [Axios](https://www.axios.com/states-face-coverings-mandatory-a0e2fe35-5b7b-458e-9d28-3f6cdb1032fb.html). This will be dummy coded.

In [50]:
dem = [0, 0, 1, 0, 3, 2, 3, 3, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 2, 3, 3, 1, 2, 0, 0, 0, 0, 1, 1, 3, 2, 3, 0, 0, 0, 0, 3, 1, 3, 0, 0, 0, 0, 0, 3, 2, 3, 0, 1, 0]
rep = [3, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 2, 3, 3, 0, 0, 0, 0, 0, 3, 2, 2, 2, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 2, 3, 3, 1, 2, 0, 0, 0, 3, 0, 3]
swing = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
mask = [1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0]

In [51]:
# repeats an array, e.gg [1,2,3] 2 times is [1,2,3,1,2,3]
def repeat(dat, times):
    temp = dat
    for i in range(int(times) - 1):
        temp = np.append(temp, dat)
    return temp

times = data.shape[0] / 50
data['dem'] = repeat(dem, times)
data['rep'] = repeat(rep, times)
data['swing'] = repeat(swing, times)
data['mask'] = repeat(mask, times)

In [52]:
data_sort = data[['Province_State', 'Last_Update', 'Confirmed']].sort_values(by = ['Province_State', 'Last_Update'])
cols = data_sort[['Province_State', 'Last_Update']].loc[data_sort['Province_State'] == 'Alabama']
new = data_sort['Confirmed'].loc[data_sort['Province_State'] == 'Alabama']
new = pd.Series(np.append(new.iloc[0], new.iloc[0:len(new) - 1]), index = data_sort.loc[data_sort['Province_State'] == 'Alabama'].index)
df = pd.concat([cols, data_sort['Confirmed'].loc[data_sort['Province_State'] == 'Alabama'] - new], axis = 1)
df.columns.values[2] = 'New_Confirmed'

for state in statenames[1:len(statenames)]:
    cols = data_sort[['Province_State', 'Last_Update']].loc[data_sort['Province_State'] == state]
    new = data_sort['Confirmed'].loc[data_sort['Province_State'] == state]
    new = pd.Series(np.append(new.iloc[0], new.iloc[0:len(new) - 1]), index = data_sort.loc[data_sort['Province_State'] == state].index)
    temp = pd.concat([cols, data_sort['Confirmed'].loc[data_sort['Province_State'] == state] - new], axis = 1)
    temp.columns.values[2] = 'New_Confirmed'
    df = df.append(temp)

data = pd.concat([data, df.sort_values(by = ['Last_Update', 'Province_State'])[['New_Confirmed']]], axis = 1)      

In [53]:
data_sort = data[['Province_State', 'Last_Update', 'Deaths']].sort_values(by = ['Province_State', 'Last_Update'])
cols = data_sort[['Province_State', 'Last_Update']].loc[data_sort['Province_State'] == 'Alabama']
new = data_sort['Deaths'].loc[data_sort['Province_State'] == 'Alabama']
new = pd.Series(np.append(new.iloc[0], new.iloc[0:len(new) - 1]), index = data_sort.loc[data_sort['Province_State'] == 'Alabama'].index)
df = pd.concat([cols, data_sort['Deaths'].loc[data_sort['Province_State'] == 'Alabama'] - new], axis = 1)
df.columns.values[2] = 'New_Deaths'

for state in statenames[1:len(statenames)]:
    cols = data_sort[['Province_State', 'Last_Update']].loc[data_sort['Province_State'] == state]
    new = data_sort['Deaths'].loc[data_sort['Province_State'] == state]
    new = pd.Series(np.append(new.iloc[0], new.iloc[0:len(new) - 1]), index = data_sort.loc[data_sort['Province_State'] == state].index)
    temp = pd.concat([cols, data_sort['Deaths'].loc[data_sort['Province_State'] == state] - new], axis = 1)
    temp.columns.values[2] = 'New_Deaths'
    df = df.append(temp)

data = pd.concat([data, df.sort_values(by = ['Last_Update', 'Province_State'])[['New_Deaths']]], axis = 1)    
data.head()

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,...,Washington,West Virginia,Wisconsin,Wyoming,dem,rep,swing,mask,New_Confirmed,New_Deaths
0,Alabama,2020-04-12 23:18:15,32.3182,-86.9023,3563.0,93.0,780.3,3470.0,75.988020,21583.0,...,0.0,0.0,0.0,0.0,0,3,0,1,0.0,0.0
1,Alaska,2020-04-12 23:18:15,61.3707,-152.4044,272.0,8.0,66.0,264.0,45.504049,8038.0,...,0.0,0.0,0.0,0.0,0,2,0,0,0.0,0.0
2,Arizona,2020-04-12 23:18:15,33.7298,-111.4312,3542.0,115.0,1378.2,3427.0,48.662422,42109.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0.0,0.0
3,Arkansas,2020-04-12 23:18:15,34.9697,-92.3731,1280.0,27.0,367.0,1253.0,49.439423,19722.0,...,0.0,0.0,0.0,0.0,0,3,0,1,0.0,0.0
4,California,2020-04-12 23:18:15,36.1162,-119.6816,22795.0,640.0,4283.8,22155.0,58.137726,190328.0,...,0.0,0.0,0.0,0.0,3,0,0,1,0.0,0.0


Next we split the data into training and testing sets.

In [54]:
train = data.drop(data[data.Last_Update.astype('datetime64[ns]') > '2020-09-01'].index)
#train.drop(columns = "Last_Update", inplace = True)
test = data.drop(data[(data.Last_Update.astype('datetime64[ns]') <= '2020-09-01')].index)
test.drop(test[(test.Last_Update.astype('datetime64[ns]') > '2020-09-27')].index, inplace = True)
#test.drop(columns = "Last_Update", inplace = True)

In [55]:
train.head()

,Province_State,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,...,Washington,West Virginia,Wisconsin,Wyoming,dem,rep,swing,mask,New_Confirmed,New_Deaths
0,Alabama,2020-04-12 23:18:15,32.3182,-86.9023,3563.0,93.0,780.3,3470.0,75.988020,21583.0,...,0.0,0.0,0.0,0.0,0,3,0,1,0.0,0.0
1,Alaska,2020-04-12 23:18:15,61.3707,-152.4044,272.0,8.0,66.0,264.0,45.504049,8038.0,...,0.0,0.0,0.0,0.0,0,2,0,0,0.0,0.0
2,Arizona,2020-04-12 23:18:15,33.7298,-111.4312,3542.0,115.0,1378.2,3427.0,48.662422,42109.0,...,0.0,0.0,0.0,0.0,1,0,0,0,0.0,0.0
3,Arkansas,2020-04-12 23:18:15,34.9697,-92.3731,1280.0,27.0,367.0,1253.0,49.439423,19722.0,...,0.0,0.0,0.0,0.0,0,3,0,1,0.0,0.0
4,California,2020-04-12 23:18:15,36.1162,-119.6816,22795.0,640.0,4283.8,22155.0,58.137726,190328.0,...,0.0,0.0,0.0,0.0,3,0,0,1,0.0,0.0


## Auto ARIMA Predictions

Uses a brute force search to find parameters with lowest AIC per state and makes predictions per state.

In [14]:
import datetime
base = datetime.date(2020, 9, 1)
date_list = [base + datetime.timedelta(days=x) for x in range(26)]
for i in range(len(date_list)):
    date_list[i] = date_list[i].strftime("%m/%d/%Y")

In [23]:
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
numdays = 26
pred = pd.DataFrame(columns = ['Province_State', 'Date', 'Confirmed'])
for state in statenames:
    temp = pd.concat([pd.Series(np.repeat(state, repeats = numdays)), pd.Series(date_list)], axis = 1)
    train_con = train_data[['Confirmed']].loc[train_data[state] == 1]
    stepwise_model = auto_arima(train_con.Confirmed, start_p=0, start_q=1,
                           max_p=10, max_q=3, m=12,
                           d=2, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    history = [x for x in train_con.Confirmed]
    predictions = list()
    for t in range(numdays):
        model = ARIMA(history, order = stepwise_model.order)
        model_fit = model.fit()
        output = model_fit.forecast()
        predictions.append(output[0])
        history.append(output[0])
    temp = pd.concat([temp, pd.Series(predictions)], axis = 1)
    temp.columns = ['Province_State', 'Date', 'Confirmed']
    pred = pred.append(temp)


Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=2007.313, Time=0.40 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=2067.439, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=2022.503, Time=0.05 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=2007.458, Time=0.11 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=2007.385, Time=0.11 sec
 ARIMA(0,2,1)(2,0,1)[12]             : AIC=2009.156, Time=0.98 sec
 ARIMA(0,2,1)(1,0,2)[12]             : AIC=2009.164, Time=1.01 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=2005.828, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=2007.283, Time=0.08 sec
 ARIMA(0,2,2)(0,0,0)[12]             : AIC=2007.365, Time=0.06 sec
 ARIMA(1,2,0)(0,0,0)[12]             : AIC=2022.867, Time=0.01 sec
 ARIMA(1,2,2)(0,0,0)[12]             : AIC=2003.776, Time=0.24 sec
 ARIMA(1,2,2)(1,0,0)[12]             : AIC=2003.131, Time=0.36 sec
 ARIMA(1,2,2)(2,0,0)[12]             : AIC=2005.023, Time=0.96 sec
 ARIMA(1,2,2)(1,0,1

C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=1224.896, Time=0.20 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=1233.001, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=1232.983, Time=0.06 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=1223.338, Time=0.08 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=1221.706, Time=0.04 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=1223.417, Time=0.06 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=1208.133, Time=0.06 sec
 ARIMA(1,2,1)(1,0,0)[12]             : AIC=1209.740, Time=0.13 sec
 ARIMA(1,2,1)(0,0,1)[12]             : AIC=1209.685, Time=0.14 sec
 ARIMA(1,2,1)(1,0,1)[12]             : AIC=1211.526, Time=0.25 sec
 ARIMA(1,2,0)(0,0,0)[12]             : AIC=1231.023, Time=0.02 sec
 ARIMA(2,2,1)(0,0,0)[12]             : AIC=1205.562, Time=0.09 sec
 ARIMA(2,2,1)(1,0,0)[12]             : AIC=1206.808, Time=0.18 sec
 ARIMA(2,2,1)(0,0,1)[12]             : AIC=1206.693, Time=0.16 sec
 ARIMA(2,2,1)(1,0,1

 ARIMA(4,2,2)(0,0,1)[12]             : AIC=1752.136, Time=0.72 sec
 ARIMA(4,2,2)(0,0,0)[12]             : AIC=1750.591, Time=0.36 sec
 ARIMA(4,2,2)(1,0,0)[12]             : AIC=1752.261, Time=0.74 sec
 ARIMA(4,2,2)(1,0,1)[12]             : AIC=1754.122, Time=0.82 sec
 ARIMA(3,2,2)(0,0,0)[12]             : AIC=1776.904, Time=0.22 sec
 ARIMA(4,2,1)(0,0,0)[12]             : AIC=1769.253, Time=0.13 sec
 ARIMA(5,2,2)(0,0,0)[12]             : AIC=1751.737, Time=0.39 sec
 ARIMA(4,2,3)(0,0,0)[12]             : AIC=1754.264, Time=0.39 sec
 ARIMA(3,2,1)(0,0,0)[12]             : AIC=1781.196, Time=0.12 sec
 ARIMA(3,2,3)(0,0,0)[12]             : AIC=1778.552, Time=0.24 sec
 ARIMA(5,2,1)(0,0,0)[12]             : AIC=1767.567, Time=0.06 sec
 ARIMA(5,2,3)(0,0,0)[12]             : AIC=1759.407, Time=0.45 sec
 ARIMA(4,2,2)(0,0,0)[12] intercept   : AIC=1752.306, Time=0.40 sec

Best model:  ARIMA(4,2,2)(0,0,0)[12]          
Total fit time: 17.675 seconds


C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\bas

...

Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=1687.039, Time=0.20 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=1707.780, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=1701.194, Time=0.08 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=1687.261, Time=0.08 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=1687.334, Time=0.07 sec
 ARIMA(0,2,1)(2,0,1)[12]             : AIC=1682.425, Time=0.42 sec
 ARIMA(0,2,1)(2,0,0)[12]             : AIC=1681.041, Time=0.27 sec
 ARIMA(0,2,0)(2,0,0)[12]             : AIC=1710.343, Time=0.14 sec
 ARIMA(1,2,1)(2,0,0)[12]             : AIC=1678.082, Time=0.38 sec
 ARIMA(1,2,1)(1,0,0)[12]             : AIC=1678.842, Time=0.14 sec
 ARIMA(1,2,1)(2,0,1)[12]             : AIC=1679.890, Time=0.57 sec
 ARIMA(1,2,1)(1,0,1)[12]             : AIC=1679.754, Time=0.18 sec
 ARIMA(1,2,0)(2,0,0)[12]             : AIC=1700.092, Time=0.23 sec
 ARIMA(2,2,1)(2,0,0)[12]             : AIC=1679.099, Time=0.45 sec
 ARIMA(1,2,2)(2,0,0

C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarnin

Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=880.865, Time=0.21 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=959.654, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=909.432, Time=0.05 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=880.307, Time=0.09 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=878.318, Time=0.02 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=880.309, Time=0.07 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=879.111, Time=0.04 sec
 ARIMA(0,2,2)(0,0,0)[12]             : AIC=879.364, Time=0.03 sec
 ARIMA(1,2,0)(0,0,0)[12]             : AIC=907.437, Time=0.02 sec
 ARIMA(1,2,2)(0,0,0)[12]             : AIC=881.020, Time=0.04 sec
 ARIMA(0,2,1)(0,0,0)[12] intercept   : AIC=880.285, Time=0.05 sec

Best model:  ARIMA(0,2,1)(0,0,0)[12]          
Total fit time: 0.645 seconds
Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=1914.791, Time=0.21 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=

C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=1393.647, Time=0.22 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=1470.592, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=1419.927, Time=0.06 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=1393.186, Time=0.08 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=1395.093, Time=0.02 sec
 ARIMA(0,2,1)(0,0,2)[12]             : AIC=1388.715, Time=0.29 sec
 ARIMA(0,2,1)(1,0,2)[12]             : AIC=1387.468, Time=0.44 sec
 ARIMA(0,2,1)(2,0,2)[12]             : AIC=1387.373, Time=0.54 sec
 ARIMA(0,2,1)(2,0,1)[12]             : AIC=1385.447, Time=0.45 sec
 ARIMA(0,2,1)(2,0,0)[12]             : AIC=1386.626, Time=0.21 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=1394.978, Time=0.08 sec
 ARIMA(0,2,0)(2,0,1)[12]             : AIC=1458.883, Time=0.35 sec
 ARIMA(1,2,1)(2,0,1)[12]             : AIC=1385.626, Time=0.49 sec
 ARIMA(0,2,2)(2,0,1)[12]             : AIC=1385.725, Time=0.47 sec
 ARIMA(1,2,0)(2,0,1

In [58]:
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
numdays = 26
pred_death = pd.DataFrame(columns = ['Province_State', 'Date', 'Deaths'])
for state in statenames:
    temp = pd.concat([pd.Series(np.repeat(state, repeats = numdays)), pd.Series(date_list)], axis = 1)
    train_death = train_data[['Deaths']].loc[train_data[state] == 1]
    print(state)
    stepwise_model = auto_arima(train_death.Deaths, start_p=0, start_q=1,
                           max_p=10, max_q=3, m=12,
                           d=2, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    history = [x for x in train_death.Deaths]
    predictions = list()
    for t in range(numdays):
        if (state == 'New York'):
            model = ARIMA(history, order = (1,2,2))
        else:
            model = ARIMA(history, order = stepwise_model.order)
        model_fit = model.fit()
        output = model_fit.forecast()
        predictions.append(output[0])
        history.append(output[0])
    temp = pd.concat([temp, pd.Series(predictions)], axis = 1)
    temp.columns = ['Province_State', 'Date', 'Deaths']
    pred_death = pred_death.append(temp)


Alabama
Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=1082.416, Time=0.27 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=1129.263, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=1119.518, Time=0.07 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=1080.565, Time=0.06 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=1078.932, Time=0.02 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=1080.638, Time=0.08 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=1074.240, Time=0.04 sec
 ARIMA(1,2,1)(1,0,0)[12]             : AIC=1076.239, Time=0.17 sec
 ARIMA(1,2,1)(0,0,1)[12]             : AIC=1076.238, Time=0.17 sec
 ARIMA(1,2,1)(1,0,1)[12]             : AIC=1078.225, Time=0.36 sec
 ARIMA(1,2,0)(0,0,0)[12]             : AIC=1117.569, Time=0.02 sec
 ARIMA(2,2,1)(0,0,0)[12]             : AIC=1073.827, Time=0.07 sec
 ARIMA(2,2,1)(1,0,0)[12]             : AIC=1075.822, Time=0.18 sec
 ARIMA(2,2,1)(0,0,1)[12]             : AIC=1075.821, Time=0.19 sec
 ARIMA(2,2,

 ARIMA(1,2,2)(2,0,1)[12]             : AIC=1444.252, Time=0.95 sec
 ARIMA(1,2,2)(1,0,1)[12]             : AIC=1459.177, Time=0.56 sec
 ARIMA(2,2,2)(2,0,0)[12]             : AIC=1443.383, Time=1.04 sec
 ARIMA(1,2,3)(2,0,0)[12]             : AIC=1441.302, Time=0.94 sec
 ARIMA(1,2,3)(1,0,0)[12]             : AIC=1448.704, Time=0.29 sec
 ARIMA(1,2,3)(2,0,1)[12]             : AIC=1443.052, Time=1.21 sec
 ARIMA(1,2,3)(1,0,1)[12]             : AIC=1447.488, Time=0.42 sec
 ARIMA(0,2,3)(2,0,0)[12]             : AIC=1449.887, Time=0.43 sec
 ARIMA(2,2,3)(2,0,0)[12]             : AIC=inf, Time=1.51 sec
 ARIMA(1,2,3)(2,0,0)[12] intercept   : AIC=1441.204, Time=1.20 sec
 ARIMA(1,2,3)(1,0,0)[12] intercept   : AIC=1449.364, Time=0.37 sec
 ARIMA(1,2,3)(2,0,1)[12] intercept   : AIC=1443.044, Time=1.50 sec
 ARIMA(1,2,3)(1,0,1)[12] intercept   : AIC=1447.479, Time=0.75 sec
 ARIMA(0,2,3)(2,0,0)[12] intercept   : AIC=1449.920, Time=0.73 sec
 ARIMA(1,2,2)(2,0,0)[12] intercept   : AIC=1441.985, Time=1.05 sec


C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Colorado
Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=1133.769, Time=0.10 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=1199.298, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=1171.961, Time=0.05 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=1132.171, Time=0.05 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=1131.618, Time=0.02 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=1132.336, Time=0.06 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=1132.616, Time=0.04 sec
 ARIMA(0,2,2)(0,0,0)[12]             : AIC=1132.608, Time=0.04 sec
 ARIMA(1,2,0)(0,0,0)[12]             : AIC=1170.003, Time=0.02 sec
 ARIMA(1,2,2)(0,0,0)[12]             : AIC=1132.842, Time=0.08 sec
 ARIMA(0,2,1)(0,0,0)[12] intercept   : AIC=1131.116, Time=0.06 sec
 ARIMA(0,2,1)(1,0,0)[12] intercept   : AIC=1131.746, Time=0.18 sec
 ARIMA(0,2,1)(0,0,1)[12] intercept   : AIC=1131.553, Time=0.15 sec
 ARIMA(0,2,1)(1,0,1)[12] intercept   : AIC=1133.210, Time=0.31 sec
 ARIMA(0,2

 ARIMA(0,2,1)(1,0,2)[12]             : AIC=inf, Time=0.97 sec
 ARIMA(0,2,0)(0,0,1)[12]             : AIC=698.716, Time=0.03 sec
 ARIMA(1,2,1)(0,0,1)[12]             : AIC=650.876, Time=0.07 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=657.931, Time=0.03 sec
 ARIMA(1,2,1)(1,0,1)[12]             : AIC=652.872, Time=0.14 sec
 ARIMA(1,2,1)(0,0,2)[12]             : AIC=652.869, Time=0.29 sec
 ARIMA(1,2,1)(1,0,0)[12]             : AIC=652.108, Time=0.10 sec
 ARIMA(1,2,1)(1,0,2)[12]             : AIC=inf, Time=0.89 sec
 ARIMA(1,2,0)(0,0,1)[12]             : AIC=688.302, Time=0.08 sec
 ARIMA(2,2,1)(0,0,1)[12]             : AIC=646.316, Time=0.14 sec
 ARIMA(2,2,1)(0,0,0)[12]             : AIC=651.474, Time=0.05 sec
 ARIMA(2,2,1)(1,0,1)[12]             : AIC=648.158, Time=0.21 sec
 ARIMA(2,2,1)(0,0,2)[12]             : AIC=647.953, Time=0.37 sec
 ARIMA(2,2,1)(1,0,0)[12]             : AIC=648.007, Time=0.12 sec
 ARIMA(2,2,1)(1,0,2)[12]             : AIC=inf, Time=1.18 sec
 ARIMA(2,2,0)(0,0,1)[1

...

C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Edwin\miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Oklahoma
Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=790.762, Time=0.13 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=840.978, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=828.323, Time=0.06 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=790.645, Time=0.09 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=797.891, Time=0.03 sec
 ARIMA(0,2,1)(0,0,2)[12]             : AIC=788.605, Time=0.28 sec
 ARIMA(0,2,1)(1,0,2)[12]             : AIC=788.803, Time=0.32 sec
 ARIMA(0,2,0)(0,0,2)[12]             : AIC=831.254, Time=0.15 sec
 ARIMA(1,2,1)(0,0,2)[12]             : AIC=785.192, Time=0.30 sec
 ARIMA(1,2,1)(0,0,1)[12]             : AIC=786.189, Time=0.14 sec
 ARIMA(1,2,1)(1,0,2)[12]             : AIC=785.313, Time=0.44 sec
 ARIMA(1,2,1)(1,0,1)[12]             : AIC=786.569, Time=0.20 sec
 ARIMA(1,2,0)(0,0,2)[12]             : AIC=823.816, Time=0.24 sec
 ARIMA(2,2,1)(0,0,2)[12]             : AIC=778.456, Time=0.34 sec
 ARIMA(2,2,1)(0,0,1)[12]

 ARIMA(1,2,1)(0,0,0)[12]             : AIC=886.334, Time=0.05 sec
 ARIMA(0,2,2)(0,0,0)[12]             : AIC=886.222, Time=0.07 sec
 ARIMA(1,2,0)(0,0,0)[12]             : AIC=933.031, Time=0.03 sec
 ARIMA(1,2,2)(0,0,0)[12]             : AIC=886.465, Time=0.08 sec
 ARIMA(0,2,1)(0,0,0)[12] intercept   : AIC=884.620, Time=0.05 sec

Best model:  ARIMA(0,2,1)(0,0,0)[12]          
Total fit time: 0.640 seconds
South Carolina
Performing stepwise search to minimize aic
 ARIMA(0,2,1)(1,0,1)[12]             : AIC=1124.925, Time=0.16 sec
 ARIMA(0,2,0)(0,0,0)[12]             : AIC=1189.640, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[12]             : AIC=1164.489, Time=0.06 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=1122.995, Time=0.07 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=1121.093, Time=0.03 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=1123.007, Time=0.05 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=1123.072, Time=0.03 sec
 ARIMA(0,2,2)(0,0,0)[12]             : AIC=1123.066, Time=0.09 se

 ARIMA(1,2,0)(1,0,0)[12]             : AIC=268.170, Time=0.05 sec
 ARIMA(0,2,1)(0,0,1)[12]             : AIC=242.247, Time=0.09 sec
 ARIMA(0,2,1)(0,0,0)[12]             : AIC=241.160, Time=0.02 sec
 ARIMA(0,2,1)(1,0,0)[12]             : AIC=242.240, Time=0.05 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=240.376, Time=0.04 sec
 ARIMA(1,2,1)(1,0,0)[12]             : AIC=242.344, Time=0.13 sec
 ARIMA(1,2,1)(0,0,1)[12]             : AIC=242.343, Time=0.13 sec
 ARIMA(1,2,1)(1,0,1)[12]             : AIC=244.342, Time=0.15 sec
 ARIMA(1,2,0)(0,0,0)[12]             : AIC=266.186, Time=0.03 sec
 ARIMA(2,2,1)(0,0,0)[12]             : AIC=240.137, Time=0.05 sec
 ARIMA(2,2,1)(1,0,0)[12]             : AIC=242.127, Time=0.13 sec
 ARIMA(2,2,1)(0,0,1)[12]             : AIC=242.127, Time=0.16 sec
 ARIMA(2,2,1)(1,0,1)[12]             : AIC=244.127, Time=0.17 sec
 ARIMA(2,2,0)(0,0,0)[12]             : AIC=264.600, Time=0.02 sec
 ARIMA(3,2,1)(0,0,0)[12]             : AIC=234.632, Time=0.07 sec
 ARIMA(3,2

 ARIMA(1,2,0)(1,0,1)[12]             : AIC=334.800, Time=0.18 sec
 ARIMA(2,2,0)(0,0,0)[12]             : AIC=325.754, Time=0.03 sec
 ARIMA(2,2,0)(1,0,0)[12]             : AIC=327.326, Time=0.06 sec
 ARIMA(2,2,0)(0,0,1)[12]             : AIC=327.298, Time=0.07 sec
 ARIMA(2,2,0)(1,0,1)[12]             : AIC=328.734, Time=0.28 sec
 ARIMA(3,2,0)(0,0,0)[12]             : AIC=325.979, Time=0.03 sec
 ARIMA(2,2,1)(0,0,0)[12]             : AIC=inf, Time=0.24 sec
 ARIMA(1,2,1)(0,0,0)[12]             : AIC=inf, Time=0.14 sec
 ARIMA(3,2,1)(0,0,0)[12]             : AIC=inf, Time=0.35 sec
 ARIMA(2,2,0)(0,0,0)[12] intercept   : AIC=327.741, Time=0.04 sec

Best model:  ARIMA(2,2,0)(0,0,0)[12]          
Total fit time: 2.266 seconds


In [92]:
pred2 = pred
pred2['Deaths'] = pred_death['Deaths']
pred2 = pred2.sort_values(by = ['Date', 'Province_State']).reset_index(drop = True)
pred2 = pred2.drop(columns = ['Province_State', 'Date'])
pred2.index.name = 'ForecastID'

In [103]:
actual = test_data[['Confirmed', 'Deaths']]
mape = np.absolute(np.array(pred2[['Confirmed', 'Deaths']]) - actual) / np.array(actual)
mape = np.sum(mape, axis = 0) / len(pred2)
#mape = np.sum(mape) / (2 * len(pred2))
mape

Confirmed    0.023014
Deaths       0.029154
dtype: float64

In [93]:
pred2.to_csv("Team1_AutoARIMA.csv")

Calculated MAPE does not seem to match the Kaggle calculation exactly for some reason.

## Archived Code

We will add a parameter for the number of days since Jan 1st, which will eventually replace the timestamps.


In [ ]:
import datetime
# add a column for days since Jan 1st(will eventually remove timestamp)
# too lazy to vectorize
days = np.zeros(len(data.index))
for i in range(len(data.index)):
    days[i] = (data['Last_Update'][i].date() - datetime.date(2020, 1, 1)).days
data['days'] = days

data.head()